In [1]:
import mysql.connector
 
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Yankejing020711"   #REPLACE THIS WITH THE PASSWORD YOU SET
)
 
print(mydb)
 
if mydb.is_connected():
    print("CONNECTION SUCCESSFUL")

CONNECTION SUCCESSFUL


In [2]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Yankejing020711", #REPLACE THIS WITH YOUR PASSWORD 
  database = "project1" #connecting to testDatabase
)

#redo table
mycursor = mydb.cursor()

In [3]:
# Q1
query = '''
SELECT N.physician_id
FROM (SELECT A.physician_id, COUNT(A.physician_id) AS physician_count
	  FROM project1.alerts  AS A
      GROUP BY A.physician_id) AS N
WHERE N.physician_count = (
	SELECT MAX(physician_count)
    FROM (
        SELECT COUNT(A.physician_id) AS physician_count
        FROM project1.alerts AS A
        GROUP BY A.physician_id
    ) AS M
    );
'''
mycursor.execute(query)
for x in mycursor:
    print(x)

('156-28-1945',)
('571-13-9020',)


In [4]:
# Q2
query = '''
SELECT DISTINCT physician_id
FROM alerts;
'''
mycursor.execute(query)
for x in mycursor:
    print(x)

('156-28-1945',)
('571-13-9020',)
('614-57-6885',)


In [5]:
# Q3
query = '''
SELECT physician_id
FROM (
	SELECT P.physician_id, COUNT(P.physician_id) AS physician_count
	FROM prescriptions AS P
	JOIN contracts AS C ON P.drug_name = C.drug_name
	JOIN companies AS CO ON C.company_id = CO.id
    WHERE CO.name = 'DRUGXO'
    GROUP BY physician_id
) AS physician_counts
WHERE physician_count = (
	SELECT MAX(physician_count)
    FROM (
		SELECT P.physician_id, COUNT(P.physician_id) AS physician_count
		FROM prescriptions AS P
		JOIN contracts AS C ON P.drug_name = C.drug_name
		JOIN companies AS CO ON C.company_id = CO.id
        WHERE CO.name = 'DRUGXO'
		GROUP BY P.physician_id
	) AS inner_counts
);
'''
mycursor.execute(query)
for x in mycursor:
    print(x)

('571-13-9020',)


In [6]:
# Q4
query = '''
SELECT 
    C.drug_name, 
    SUM(C.price) / SUM(C.quantity) AS price_per_unit,
    (SELECT SUM(price) / SUM(quantity) 
     FROM contracts 
     WHERE drug_name = C.drug_name) AS avg_price
FROM contracts AS C
JOIN companies AS CO ON C.company_id = CO.id
WHERE CO.name = 'PHARMASEE'
GROUP BY C.drug_name;
'''
mycursor.execute(query)
for x in mycursor:
    print(x)

('Glucozepam Amcipentin', Decimal('1.5900000'), Decimal('1.5900000'))


In [7]:
# Q5
query = '''
SELECT P.drug_name, PF.pharmacy_id, (((PF.cost/P.quantity)-(C.price/C.quantity))/(C.price/C.quantity))*100 AS markup
FROM prescriptions AS P
LEFT JOIN pharmacy_fills AS PF ON PF.prescription_id = P.id
LEFT JOIN contracts AS C ON (C.pharmacy_id, C.drug_name) = (PF.pharmacy_id, P.drug_name);
'''
mycursor.execute(query)
for x in mycursor:
    print(x)

('Avafoxin', '8', None)
('Cleotrana', '2', None)
('Dantopex Quixilinum', '12', None)
('Cleotrana', '14', None)
('Avafoxin', '6', None)
('Quixiposide', '7', None)
('Avafoxin', '9', None)
('Kanulin', '4', Decimal('917.85714285714'))
('Cleotrana', '1', None)
('Primalovir', '1', None)
('Glucozepam Amcipentin', '3', None)
('Olanzanafine', '10', None)
('Primalovir', '7', None)
('Abnazole Toleluble', '5', None)
('Avafoxin', '5', None)
('Glucozepam Amcipentin', '11', None)


In [8]:
# Q6
query = '''
SELECT D.name, AVG(DATEDIFF(PF.date, P.date)) AS avg_day_between
FROM drugs AS D
LEFT JOIN prescriptions AS P ON D.name = P.drug_name
LEFT JOIN pharmacy_fills AS PF ON P.id = PF.prescription_id
GROUP BY D.name;
'''
mycursor.execute(query)
for x in mycursor:
    print(x)

('Abnazole Toleluble', Decimal('4.0000'))
('Avafoxin', Decimal('1.7500'))
('Cleotrana', Decimal('1.3333'))
('Dantopex Quixilinum', Decimal('2.0000'))
('Divisporine Acetaclotide', None)
('Glucozepam Amcipentin', Decimal('1.5000'))
('Kanulin', Decimal('0.0000'))
('Olanzanafine', Decimal('2.0000'))
('Primalovir', Decimal('-13.5000'))
('Quixiposide', Decimal('2.0000'))


In [9]:
# Q7
query = '''
SELECT DISTINCT P.id AS pharmacy_id, D.name AS drug_name
FROM pharmacies P
CROSS JOIN drugs D
LEFT JOIN (
	SELECT PR.drug_name, PF.pharmacy_id
	FROM pharmacy_fills PF
	JOIN prescriptions PR ON PF.prescription_id = PR.id) AS PF 
    ON P.id = PF.pharmacy_id AND D.name = PF.drug_name
	WHERE 
		NOT EXISTS (
        #Filled x drug at x pharmacy
        SELECT 1
        FROM pharmacy_fills PF
        WHERE PF.pharmacy_id = P.id AND PF.drug_name = D.name
    )
AND EXISTS (
    #Existed prescription for the drug
    SELECT 1
    FROM prescriptions PR
    WHERE PR.drug_name = D.name
);
'''
mycursor.execute(query)
for x in mycursor:
    print(x)

('1', 'Abnazole Toleluble')
('1', 'Avafoxin')
('1', 'Dantopex Quixilinum')
('1', 'Glucozepam Amcipentin')
('1', 'Kanulin')
('1', 'Olanzanafine')
('1', 'Quixiposide')
('10', 'Abnazole Toleluble')
('10', 'Avafoxin')
('10', 'Cleotrana')
('10', 'Dantopex Quixilinum')
('10', 'Glucozepam Amcipentin')
('10', 'Kanulin')
('10', 'Primalovir')
('10', 'Quixiposide')
('11', 'Abnazole Toleluble')
('11', 'Avafoxin')
('11', 'Cleotrana')
('11', 'Dantopex Quixilinum')
('11', 'Kanulin')
('11', 'Olanzanafine')
('11', 'Primalovir')
('11', 'Quixiposide')
('12', 'Abnazole Toleluble')
('12', 'Avafoxin')
('12', 'Cleotrana')
('12', 'Glucozepam Amcipentin')
('12', 'Kanulin')
('12', 'Olanzanafine')
('12', 'Primalovir')
('12', 'Quixiposide')
('13', 'Abnazole Toleluble')
('13', 'Avafoxin')
('13', 'Cleotrana')
('13', 'Dantopex Quixilinum')
('13', 'Glucozepam Amcipentin')
('13', 'Kanulin')
('13', 'Olanzanafine')
('13', 'Primalovir')
('13', 'Quixiposide')
('14', 'Abnazole Toleluble')
('14', 'Avafoxin')
('14', 'Dantope

In [10]:
# ALERT TABLE
query = '''
SELECT * FROM alerts;
'''
mycursor.execute(query)
for x in mycursor:
    print(x)

('501-47-2038', '156-28-1945', datetime.date(2023, 9, 22), 'Avafoxin', 'Kanulin')
('501-47-2038', '156-28-1945', datetime.date(2023, 9, 22), 'Cleotrana', 'Avafoxin')
('501-47-2038', '156-28-1945', datetime.date(2023, 9, 22), 'Cleotrana', 'Kanulin')
('303-13-5928', '571-13-9020', datetime.date(2023, 5, 24), 'Olanzanafine', 'Primalovir')
('303-13-5928', '571-13-9020', datetime.date(2023, 5, 24), 'Primalovir', 'Abnazole Toleluble')
('303-13-5928', '571-13-9020', datetime.date(2023, 6, 22), 'Olanzanafine', 'Glucozepam Amcipentin')
('478-34-0781', '614-57-6885', datetime.date(2023, 9, 17), 'Avafoxin', 'Quixiposide')
